In [1]:
from __future__ import unicode_literals
import classifiers
import re
import nltk
from Politweet import get_tweets, get_transcript
import ratings
from sentiment import polarity_train, classify, prob_classify, plus_df, minus_df
import pandas as pd
pd.set_option('display.max_colwidth', 1200)

In [2]:
tweets = get_tweets("./datasets/tweets.tsv")
transcript = get_transcript("./datasets/transcript.csv")
todo = set(tweets.index)


In [3]:
tweets.to_pickle('./datasets/tweets.pickle')

In [4]:
polarity = polarity_train(tweets)
polarity_minus = set(i for i,_ in classify(minus_df(tweets), polarity))
polarity_plus = set(i for i,_ in classify(plus_df(tweets), polarity))
polarities = tweets.reindex(polarity_minus | polarity_plus).sort()
print len(polarities), "- polarity"
todo -= set(polarities.index)

482 - polarity


/Library/Python/2.7/site-packages/pandas/core/strings.py:189: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  " groups, use str.extract.", UserWarning)


In [5]:
others = ratings.all(tweets, ratings.OTHER)
print len(others), "- marked as OTHERS"
todo -= set(others.index)

151 - marked as OTHERS


In [6]:
print len(todo), "- untagged"

2605 - untagged


In [7]:
left = tweets.reindex(todo).sort()
left[:1]

,pub.date.GMT,content,author.name,author.nickname,rating.1,rating.2,rating.3,rating.4,rating.5,rating.6,rating.7,rating.8,clean,tokens
tweet.id,,,,,,,,,,,,,,
936466790,9/27/08 1:00,Jim Lehrer just directed the debate audience ... 30 seconds ... #tweetdebate,B_E_X,Becky L,4,2,4,NaN,NaN,NaN,NaN,NaN,"[{u'lemma': u'jim', u'token': u'jim', u'pos': u'NN'}, {u'lemma': u'lehrer', u'token': u'lehrer', u'pos': u'NN'}, {u'lemma': u'direct', u'token': u'directed', u'pos': u'VBN'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'audience', u'token': u'audience', u'pos': u'NN'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'30', u'token': u'30', u'pos': u'CD'}, {u'lemma': u'second', u'token': u'seconds', u'pos': u'NNS'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}]","[jim, lehrer, direct, debate, audience, second, tweetdebate]"


In [8]:
left[["content","tokens", "clean"]]

,content,tokens,clean
tweet.id,,,
936466790,Jim Lehrer just directed the debate audience ... 30 seconds ... #tweetdebate,"[jim, lehrer, direct, debate, audience, second, tweetdebate]","[{u'lemma': u'jim', u'token': u'jim', u'pos': u'NN'}, {u'lemma': u'lehrer', u'token': u'lehrer', u'pos': u'NN'}, {u'lemma': u'direct', u'token': u'directed', u'pos': u'VBN'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'audience', u'token': u'audience', u'pos': u'NN'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'30', u'token': u'30', u'pos': u'CD'}, {u'lemma': u'second', u'token': u'seconds', u'pos': u'NNS'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}]"
936466992,Here we go. #tweetdebate,[tweetdebate],"[{u'lemma': u'go', u'token': u'go', u'pos': u'VBP'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}]"
936467521,#tweetdebate letting mom/dad follow debate on CNN while tweet scoring. I will sleep on the sofa with them & wait for the screams GO Obama,"[tweetdebate, let, momdad, follow, debate, cnn, tweet, scoring, sleep, sofa, wait, scream, obama]","[{u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}, {u'lemma': u'let', u'token': u'letting', u'pos': u'VBG'}, {u'lemma': u'momdad', u'token': u'momdad', u'pos': u'JJ'}, {u'lemma': u'follow', u'token': u'follow', u'pos': u'NN'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'cnn', u'token': u'cnn', u'pos': u'NN'}, {u'lemma': u'tweet', u'token': u'tweet', u'pos': u'NN'}, {u'lemma': u'scoring', u'token': u'scoring', u'pos': u'NN'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'sleep', u'token': u'sleep', u'pos': u'VB'}, {u'lemma': u'sofa', u'token': u'sofa', u'pos': u'NN'}, {u'lemma': u'', u'token': u'', u'pos': u'CC'}, {u'lemma': u'wait', u'token': u'wait', u'pos': u'VB'}, {u'lemma': u'scream', u'token': u'screams', u'pos': u'NNS'}, {u'lemma': u'go', u'token': u'go', u'pos': u'VBP'}, {u'lemma': u'obama', u'token': u'obama', u'pos': u'NN'}]"
936467712,what is an appropriately bi-partisan food to bring to a debate party? #tweetdebate,"[appropriately, bipartisan, food, bring, debate, party, tweetdebate]","[{u'lemma': u'appropriately', u'token': u'appropriately', u'pos': u'RB'}, {u'lemma': u'bipartisan', u'token': u'bipartisan', u'pos': u'JJ'}, {u'lemma': u'food', u'token': u'food', u'pos': u'NN'}, {u'lemma': u'bring', u'token': u'bring', u'pos': u'VB'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'party', u'token': u'party', u'pos': u'NN'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}]"
936468736,@current I've got the live stream going #current....awesome!,"[current, get, live, stream, current, awesome]","[{u'lemma': u'', u'token': u'', u'pos': u'IN'}, {u'lemma': u'current', u'token': u'current', u'pos': u'JJ'}, {u'lemma': u've', u'token': u've', u'pos': u'VBP'}, {u'lemma': u'get', u'token': u'got', u'pos': u'VBN'}, {u'lemma': u'live', u'token': u'live', u'pos': u'JJ'}, {u'lemma': u'stream', u'token': u'stream', u'pos': u'NN'}, {u'lemma': u'go', u'token': u'going', u'pos': u'VBG'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'current', u'token': u'current', u'pos': u'JJ'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'awesome', u'token': u'awesome', u'pos': u'JJ'}, {u'lemma': u'', u'token': u'', u'pos': u''}]"
936468824,watching the debate on current TV in SF. let's go! #tweetdebate,"[watch, debate, current, let, tweetdebate]","[{u'lemma': u'watch', u'token': u'watching', u'pos': u'VBG'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'current', u'token': u'curre

In [12]:
for i, tweet in tweets[:3].iterrows():
    print [token for token in tweet["tokens"]]
    
tokens = [" ".join(tweet["tokens"]) for i,tweet in tweets.iterrows()]
tokens[:1]

[u'watch', u'tweetdebate', u'drink', u'wait', u'start', u'cringing', u'mccain', u'blunder']
[u'ahg3', u'michdot', u'yeah', u'slime', u'actually', u'second', u'choice', u'say', u'first', u'one', u'okay', u'roll']
[u'prepare', u'heart', u'attack', u'tweetdebate']


[u'watch tweetdebate drink wait start cringing mccain blunder']

In [31]:
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
count = Counter([token for i, tweet in tweets.iterrows() for token in tweet["tokens"]])

vectorizer = TfidfVectorizer(min_df=1, max_df=1.0, use_idf=True, lowercase=False)
 
tfidf_model = vectorizer.fit_transform(tokens)
tfidf_model

<3238x4782 sparse matrix of type '<type 'numpy.float64'>'
	with 25827 stored elements in Compressed Sparse Row format>

In [38]:


inv_map = {v: k for k, v in vectorizer.vocabulary_.items()}

wf = pd.DataFrame([(k,v) for k,v in vectorizer.vocabulary_.items()]).sort(1, ascending=False)
wf

,0,1
2952,zoom,4781
3086,zonk,4780
4184,zombie,4779
4406,zogby,4778
938,zod,4777
2242,zinger,4776
1261,zing,4775
1310,zimafactcheckwire,4774
449,zima9676b6,4773
790,zima,4772
